### RandomizedSearchCV にリトライ、ImageDataGenerator/model.fit で実行


In [1]:
import os
import re
import datetime
import random
import numpy as np
from PIL import Image
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras.applications import Xception
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/drive')


def generate_data(x, t, num=5):
  datagen = ImageDataGenerator(
    #rotation_range=180,     # randomly rotate images in the range
    zoom_range=0.2,         # randomly zoom image
    width_shift_range=0.2,  # randomly shift images horizontally
    height_shift_range=0.2, # randomly shift images vertically
    horizontal_flip=True,   # randomly flip images horizontally
    #vertical_flip=True      # randomly flip images vertically
  )

  x_list = []
  t_list = []

  x_array = np.expand_dims(x, axis=0)

  datagen.fit(x_array)

  iter = datagen.flow(x_array, batch_size=1)

  for i in range(num):
    ret = iter.next()
    #print(ret.shape, ret.dtype)
    x_list.append(ret[0])
    t_list.append(t)

  return x_list, t_list


def prepare_data(gen_num=5):
  x, t = [], []

  categorized_dir_paths = glob('/content/drive/My Drive/kikagaku/novelapp/data/images/categorized-2/*')

  for dir_path in categorized_dir_paths:
    category_id = os.path.basename(dir_path)
    image_paths = glob(os.path.join(dir_path, '*'))
    print(datetime.datetime.now().isoformat(), 'Category', category_id, ':', len(image_paths))

    for i, p in enumerate(image_paths):
      book_id = re.sub(r'(_.*$)', '', os.path.basename(p))

      if i % 30 == 0:
          print(datetime.datetime.now().isoformat(), 'Image:', i, book_id, category_id)

      try:
          img = Image.open(p)
          img_resize = img.resize((229, 229))
          img_np = np.array(img_resize) / 255.0

          x.append(img_np)
          t.append(category_id)

          x_gen, t_gen = generate_data(img_np, category_id, gen_num)
          x.extend(x_gen)
          t.extend(t_gen)

      except Exception as e:
          print(datetime.datetime.now().isoformat(), 'Error:', e)

  return x, t


def reset_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

reset_seed(0)


x, t = prepare_data(5)
print(len(x), len(t))

x = np.array(x).astype('f')
t = np.array(t).astype('f')
print(x.shape, t.shape)

x_train, x_val, t_train, t_val = train_test_split(x, t, test_size=0.3, random_state=0)
print(x_train.shape, x_val.shape)

category_count = len(np.unique(t))
print(np.unique(t))
print(category_count)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2020-06-16T12:03:40.666824 Category 0 : 117
2020-06-16T12:03:40.667941 Image: 0 1015 0
2020-06-16T12:03:42.744371 Image: 30 1228 0
2020-06-16T12:03:44.732058 Image: 60 1106 0
2020-06-16T12:03:46.705211 Image: 90 1129 0
2020-06-16T12:03:48.519689 Category 1 : 119
2020-06-16T12:03:48.521085 Image: 0 3636 1
2020-06-16T12:03:50.487426 Image: 30 3722 1
2020-06-16T12:03:52.459231 Image: 60 3832 1
2020-06-16T12:03:54.431204 Image: 90 3777 1
2020-06-16T12:03:56.358521 Category 2 : 129
2020-06-16T12:03:56.358866 Image: 0 3601 2
2020-06-16T12:03:58.344299 Image: 30 3884 2
2020-06-16T12:04:00.295144 Image: 60 3778 2
2020-06-16T12:04:02.259061 Image: 90 3853 2
2020-06-16T12:04:04.273330 Image: 120 1502 2
2020-06-16T12:04:04.888582 Category 3 : 111
2020-06-16T12:04:04.889780 Image: 0 3312 3
2020-06-16T12:04:06.877620 Image: 30 2350 3
2020-06-16T12:04:08.881456 Image: 60 3

In [0]:
def build_model(units, dropout, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

In [20]:
print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 32]#, 128, 512]
dropout = [0.01, 0.05]#, 0.1, 0.2]
lr = [0.01, 0.007]#, 0.002, 0.001, 0.0008]

param_dict = dict(units=units, 
                  dropout=dropout,
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 3
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T09:33:14.045594 Start
63/63 [==============================] - 28s 450ms/step - loss: 2.2441 - accuracy: 0.4762
2020-06-16T09:39:27.810777 End


In [21]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,2,0,1
mean_fit_time,27.8332,33.5459,27.6242
std_fit_time,0.519631,8.17001,0.141354
mean_score_time,7.50363,8.09586,7.49678
std_score_time,0.0286385,0.447563,0.0171645
param_units,32,10,10
param_lr,0.01,0.007,0.007
param_dropout,0.01,0.05,0.01
params,"{'units': 32, 'lr': 0.01, 'dropout': 0.01}","{'units': 10, 'lr': 0.007, 'dropout': 0.05}","{'units': 10, 'lr': 0.007, 'dropout': 0.01}"
split0_test_score,0.503748,0.473763,0.364318
split1_test_score,0.424925,0.459459,0.444444


In [23]:
rand.best_params_

{'dropout': 0.01, 'lr': 0.01, 'units': 32}

In [24]:
print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 32]#, 128, 512]
dropout = [0.01, 0.1]#, 0.1, 0.2]
lr = [0.01, 0.05]#, 0.002, 0.001, 0.0008]

param_dict = dict(units=units, 
                  dropout=dropout,
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 5
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T09:44:35.943552 Start
63/63 [==============================] - 27s 432ms/step - loss: 2.1409 - accuracy: 0.4957
2020-06-16T09:54:24.128886 End


In [25]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,2,0,1
mean_fit_time,31.8318,31.4297,31.8098
std_fit_time,0.473111,0.86376,0.924345
mean_score_time,5.18085,5.03552,4.91317
std_score_time,0.503564,0.258842,0.0387963
param_units,32,10,10
param_lr,0.01,0.05,0.05
param_dropout,0.01,0.1,0.01
params,"{'units': 32, 'lr': 0.01, 'dropout': 0.01}","{'units': 10, 'lr': 0.05, 'dropout': 0.1}","{'units': 10, 'lr': 0.05, 'dropout': 0.01}"
split0_test_score,0.5575,0.2425,0.2425
split1_test_score,0.5275,0.245,0.2575


In [26]:
rand.best_params_

{'dropout': 0.01, 'lr': 0.01, 'units': 32}

In [0]:
print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 128, 512]
dropout = [0.01, 0.05, 0.001]
lr = [0.008, 0.005, 0.001]

param_dict = dict(units=units, 
                  dropout=dropout,
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 3
n_iter = 4
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T10:18:02.732601 Start
21/21 [==============================] - 6s 301ms/step - loss: 1.2632 - accuracy: 0.4918


In [15]:
print(datetime.datetime.now().isoformat(), 'Start')

units = [128, 512]
dropout = [0.01, 0.05, 0.001]
lr = [0.008, 0.005, 0.001]

param_dict = dict(units=units, 
                  dropout=dropout,
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 3
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T10:28:33.626300 Start
63/63 [==============================] - 29s 465ms/step - loss: 2.2231 - accuracy: 0.4817
2020-06-16T10:34:51.842289 End


In [16]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,0,2,1
mean_fit_time,34.2334,27.7043,27.8997
std_fit_time,8.13243,0.338548,0.123593
mean_score_time,8.12587,7.69728,7.54922
std_score_time,0.423524,0.231749,0.0376985
param_units,512,128,128
param_lr,0.008,0.005,0.008
param_dropout,0.01,0.05,0.05
params,"{'units': 512, 'lr': 0.008, 'dropout': 0.01}","{'units': 128, 'lr': 0.005, 'dropout': 0.05}","{'units': 128, 'lr': 0.008, 'dropout': 0.05}"
split0_test_score,0.361319,0.445277,0.46027
split1_test_score,0.459459,0.472973,0.406907


In [17]:
def build_model(units, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 1024]
#dropout = [0.01, 0.05, 0.001]
lr = [0.008, 0.007, 0.006]

param_dict = dict(units=units, 
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T10:41:24.186497 Start
63/63 [==============================] - 29s 466ms/step - loss: 2.2272 - accuracy: 0.4742
2020-06-16T10:49:39.248781 End


In [18]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,0,2,1
mean_fit_time,32.4814,32.7692,30.4222
std_fit_time,1.47656,0.911695,0.315345
mean_score_time,6.37329,6.07093,5.90325
std_score_time,0.360492,0.0284349,0.0398427
param_units,1024,1024,10
param_lr,0.006,0.008,0.007
params,"{'units': 1024, 'lr': 0.006}","{'units': 1024, 'lr': 0.008}","{'units': 10, 'lr': 0.007}"
split0_test_score,0.492,0.55,0.39
split1_test_score,0.518,0.498,0.452
split2_test_score,0.494,0.496,0.4


In [0]:
def build_model(units, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

units = [1024, 4096]
#dropout = [0.01, 0.05, 0.001]
lr = [0.003, 0.005, 0.0005]

param_dict = dict(units=units, 
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T10:52:52.043979 Start


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[100352,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul]

  FitFailedWarning)


16/16 [==============================] - 5s 295ms/step - loss: 1.2573 - accuracy: 0.4770


In [14]:
def build_model(units, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

units = [1024, 4096]
#dropout = [0.01, 0.05, 0.001]
lr = [0.003, 0.005, 0.0005]

param_dict = dict(units=units, 
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T10:58:27.414085 Start
16/16 [==============================] - 5s 318ms/step - loss: 1.2617 - accuracy: 0.4540


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[100352,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

  FitFailedWarning)


16/16 [==============================] - 6s 347ms/step - loss: 1.6128 - accuracy: 0.4148


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[100352,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul]

  FitFailedWarning)


63/63 [==============================] - 30s 471ms/step - loss: 2.1050 - accuracy: 0.4787
2020-06-16T11:05:24.067823 End


In [15]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,1,0,2
mean_fit_time,31.9918,35.428,15.6181
std_fit_time,0.839304,13.5952,0.281506
mean_score_time,6.31963,5.01523,0
std_score_time,0.346944,2.91463,0
param_units,1024,4096,4096
param_lr,0.005,0.0005,0.003
params,"{'units': 1024, 'lr': 0.005}","{'units': 4096, 'lr': 0.0005}","{'units': 4096, 'lr': 0.003}"
split0_test_score,0.508,0.372,NaN
split1_test_score,0.466,0.362,NaN
split2_test_score,0.524,0.454,NaN


In [16]:
def build_model(units, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

units = [2048, 3000]
#dropout = [0.01, 0.05, 0.001]
lr = [0.003, 0.005, 0.0005]

param_dict = dict(units=units, 
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train, batch_size=16, epochs=5)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T11:12:08.462765 Start


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[100352,3000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul]

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[100352,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

  FitFailedWarning)


ResourceExhaustedError: ignored

In [17]:
def build_model(units, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 1024]
#dropout = [0.01, 0.05, 0.001]
lr = [0.003, 0.005, 0.0005]

param_dict = dict(units=units, 
                  lr=lr)

model = KerasClassifier(build_fn = build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train, batch_size=16, epochs=5, verbose=2)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T11:16:51.222121 Start


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[100352,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[1,1,728,728] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: E

ResourceExhaustedError: ignored

In [15]:
def build_model(units, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 32]
#dropout = [0.01, 0.05, 0.001]
lr = [0.003, 0.005, 0.0005]

param_dict = dict(units=units, 
                  lr=lr)

model = KerasClassifier(build_fn=build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train, batch_size=16)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T11:31:11.577366 Start
125/125 [==============================] - 30s 239ms/step - loss: 2.2238 - accuracy: 0.4022
2020-06-16T11:39:24.541062 End


In [16]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,1,2,0
mean_fit_time,31.4207,32.668,31.4275
std_fit_time,0.84345,2.31685,0.821995
mean_score_time,5.90124,5.91883,6.11731
std_score_time,0.0117712,0.0253433,0.376885
param_units,10,32,32
param_lr,0.005,0.003,0.0005
params,"{'units': 10, 'lr': 0.005}","{'units': 32, 'lr': 0.003}","{'units': 32, 'lr': 0.0005}"
split0_test_score,0.484,0.414,0.402
split1_test_score,0.514,0.472,0.394
split2_test_score,0.478,0.464,0.426


In [0]:
def build_model(lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

#units = [10, 32]
#dropout = [0.01, 0.05, 0.001]
batch_size = [64, 128]
lr = [0.004, 0.005]

param_dict = dict(batch_size=batch_size, 
                  lr=lr)

model = KerasClassifier(build_fn=build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T11:44:04.249103 Start
4/4 [==============================] - 6s 2s/step - loss: 1.7329 - accuracy: 0.2886


In [2]:
def build_model(lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

#units = [10, 32]
#dropout = [0.01, 0.05, 0.001]
batch_size = [64, 128]
lr = [0.004, 0.005]

param_dict = dict(batch_size=batch_size, 
                  lr=lr)

model = KerasClassifier(build_fn=build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T11:51:58.057789 Start
32/32 [==============================] - 27s 851ms/step - loss: 2.2443 - accuracy: 0.3857
2020-06-16T12:00:14.706960 End


In [3]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,2,0,1
mean_fit_time,30.9664,35.4666,28.9277
std_fit_time,1.46325,9.46001,0.258855
mean_score_time,6.60347,7.22402,5.79859
std_score_time,0.804497,1.53593,0.182179
param_lr,0.005,0.004,0.005
param_batch_size,64,128,128
params,"{'lr': 0.005, 'batch_size': 64}","{'lr': 0.004, 'batch_size': 128}","{'lr': 0.005, 'batch_size': 128}"
split0_test_score,0.402,0.376,0.29
split1_test_score,0.412,0.416,0.348
split2_test_score,0.424,0.368,0.366


In [2]:
def build_model(lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

#units = [10, 32]
#dropout = [0.01, 0.05, 0.001]
batch_size = [16, 32]
lr = [0.004, 0.003]

param_dict = dict(batch_size=batch_size, 
                  lr=lr)

model = KerasClassifier(build_fn=build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T12:07:23.161986 Start
125/125 [==============================] - 30s 238ms/step - loss: 2.1703 - accuracy: 0.4327
2020-06-16T12:15:43.429372 End


In [3]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,2,0,1
mean_fit_time,31.8609,34.7119,30.0617
std_fit_time,0.972584,7.65719,0.299168
mean_score_time,6.37336,6.28438,5.88934
std_score_time,0.23836,0.446973,0.0227969
param_lr,0.003,0.004,0.003
param_batch_size,16,32,32
params,"{'lr': 0.003, 'batch_size': 16}","{'lr': 0.004, 'batch_size': 32}","{'lr': 0.003, 'batch_size': 32}"
split0_test_score,0.456,0.476,0.372
split1_test_score,0.478,0.448,0.314
split2_test_score,0.494,0.406,0.364


In [4]:
def build_model(lr, opt_code):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(category_count, activation='softmax'))

  if opt_code == 'adag':
    optimizer = keras.optimizers.Adagrad(lr=lr)
  elif opt_code == 'adam':
    optimizer = keras.optimizers.Adam(lr=lr)
  else:
    optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


print(datetime.datetime.now().isoformat(), 'Start')

#units = [10, 32]
#dropout = [0.01, 0.05, 0.001]
batch_size = [28, 36]
lr = [0.0045, 0.0055]
opt_code=['adag', 'adam']

param_dict = dict(opt_code=opt_code,
                  batch_size=batch_size,
                  lr=lr)

model = KerasClassifier(build_fn=build_model, verbose=1)

cv = 4
n_iter = 3
random_state = 0

rand = RandomizedSearchCV(estimator=model,
                          param_distributions=param_dict,
                          n_iter=n_iter,
                          cv=cv,
                          random_state=random_state,
                          n_jobs=1,
                          pre_dispatch=1)

rand.fit(x_train, t_train)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-16T12:21:48.324478 Start
72/72 [==============================] - 27s 374ms/step - loss: 1.8933 - accuracy: 0.2876
2020-06-16T12:29:53.376352 End


In [5]:
pd.DataFrame(rand.cv_results_).sort_values('rank_test_score').T

,1,0,2
mean_fit_time,31.0632,31.7188,31.3413
std_fit_time,0.862532,1.21674,0.96661
mean_score_time,5.89555,6.17866,5.86052
std_score_time,0.0256102,0.510569,0.0302831
param_opt_code,adag,adag,adam
param_lr,0.0055,0.0055,0.0045
param_batch_size,28,36,28
params,"{'opt_code': 'adag', 'lr': 0.0055, 'batch_size...","{'opt_code': 'adag', 'lr': 0.0055, 'batch_size...","{'opt_code': 'adam', 'lr': 0.0045, 'batch_size..."
split0_test_score,0.272,0.29,0.266
split1_test_score,0.336,0.236,0.25
